In [ ]:
import pandas as pd
import os
import re
from pathlib import Path
from typing import List, Dict, Optional
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有可配置参数"""
    INPUT_DIR = Path(r'E:\A智网\业扩分析\数据准备\新装增容数据')
    OUTPUT_FILE = Path(r'E:\A智网\业扩分析\8月分析\7月业扩月度报告\25年7月业扩报告_新装增容业扩.xlsx') # 建议新文件名以防覆盖
    FILENAME_PATTERN = r'^\d{6}\.xlsx$'

    TARGET_SHEET_NAME = '国网湖北省电力有限公司'
    SKIP_ROWS = 3
    HEADER_ROWS_COUNT = 4

    ### 重构/新增开始 ###

    # --- 核心指标的最终命名 ---
    # Part 1: 从源文件直接提取的指标 (栏目 13-24)

    # 1.1 完成新装增容 (合计)
    COL_TOTAL_ALL_COUNT     = '完成新装增容_户数'
    COL_TOTAL_ALL_CAPACITY  = '完成新装增容_容量'

    # 1.2 完成新装增容 (高压)
    COL_TOTAL_HV_COUNT      = '高压完成新装增容_户数'
    COL_TOTAL_HV_CAPACITY   = '高压完成新装增容_容量'

    # 1.3 完成新装增容 (低压)
    COL_TOTAL_LV_COUNT      = '低压完成新装增容_户数'
    COL_TOTAL_LV_CAPACITY   = '低压完成新装增容_容量'

    # 1.4 完成新装 (合计)
    COL_NEW_ALL_COUNT       = '完成新装_户数'
    COL_NEW_ALL_CAPACITY    = '完成新装_容量'

    # 1.5 完成新装 (高压)
    COL_NEW_HV_COUNT        = '高压完成新装_户数'
    COL_NEW_HV_CAPACITY     = '高压完成新装_容量'

    # 1.6 完成新装 (低压)
    COL_NEW_LV_COUNT        = '低压完成新装_户数'
    COL_NEW_LV_CAPACITY     = '低压完成新装_容量'


    # Part 2: 通过计算生成的衍生指标 (增容 = 新装增容 - 新装)

    # 2.1 完成增容 (合计)
    COL_INC_ALL_COUNT       = '完成增容_户数'
    COL_INC_ALL_CAPACITY    = '完成增容_容量'

    # 2.2 完成增容 (高压)
    COL_INC_HV_COUNT        = '高压完成增容_户数'
    COL_INC_HV_CAPACITY     = '高压完成增容_容量'

    # 2.3 完成增容 (低压)
    COL_INC_LV_COUNT        = '低压完成增容_户数'
    COL_INC_LV_CAPACITY     = '低压完成增容_容量'

    ### 重构/新增结束 ###

    @staticmethod
    def get_sheets_to_create() -> Dict[str, str]:
        """
        定义要创建的所有工作表。
        字典的键 (key) 是最终Excel中的工作表名称。
        字典的值 (value) 是 long_df 中对应的数据列名。
        此方法现在将返回所有18个指标的工作表定义。
        """
        ### 重构/新增开始 ###
        sheets = {
            # 新装增容 (合计)
            Config.COL_TOTAL_ALL_COUNT:    Config.COL_TOTAL_ALL_COUNT,
            Config.COL_TOTAL_ALL_CAPACITY: Config.COL_TOTAL_ALL_CAPACITY,
            # 新装增容 (高压)
            Config.COL_TOTAL_HV_COUNT:     Config.COL_TOTAL_HV_COUNT,
            Config.COL_TOTAL_HV_CAPACITY:  Config.COL_TOTAL_HV_CAPACITY,
            # 新装增容 (低压)
            Config.COL_TOTAL_LV_COUNT:     Config.COL_TOTAL_LV_COUNT,
            Config.COL_TOTAL_LV_CAPACITY:  Config.COL_TOTAL_LV_CAPACITY,

            # 新装 (合计)
            Config.COL_NEW_ALL_COUNT:      Config.COL_NEW_ALL_COUNT,
            Config.COL_NEW_ALL_CAPACITY:   Config.COL_NEW_ALL_CAPACITY,
            # 新装 (高压)
            Config.COL_NEW_HV_COUNT:       Config.COL_NEW_HV_COUNT,
            Config.COL_NEW_HV_CAPACITY:    Config.COL_NEW_HV_CAPACITY,
            # 新装 (低压)
            Config.COL_NEW_LV_COUNT:       Config.COL_NEW_LV_COUNT,
            Config.COL_NEW_LV_CAPACITY:    Config.COL_NEW_LV_CAPACITY,

            # 增容 (合计) - 衍生
            Config.COL_INC_ALL_COUNT:      Config.COL_INC_ALL_COUNT,
            Config.COL_INC_ALL_CAPACITY:   Config.COL_INC_ALL_CAPACITY,
            # 增容 (高压) - 衍生
            Config.COL_INC_HV_COUNT:       Config.COL_INC_HV_COUNT,
            Config.COL_INC_HV_CAPACITY:    Config.COL_INC_HV_CAPACITY,
            # 增容 (低压) - 衍生
            Config.COL_INC_LV_COUNT:       Config.COL_INC_LV_COUNT,
            Config.COL_INC_LV_CAPACITY:    Config.COL_INC_LV_CAPACITY,
        }
        return sheets
        ### 重构/新增结束 ###

# ==============================================================================
# 2. 提取与解析层 (Extraction & Parsing Layer)
# ==============================================================================
def robust_to_numeric(series: pd.Series) -> pd.Series:
    """
    一个极其健壮的函数，用于清洗并转换一列可能包含文本格式数字的数据。
    它能处理千位分隔符(,)、首尾空格等常见问题。
    """
    s = series.astype(str).str.replace(',', '', regex=False).str.strip()
    return pd.to_numeric(s, errors='coerce')


def process_single_file(file_path: Path) -> Optional[pd.DataFrame]:
    """读取并处理单个Excel文件，返回一个干净的DataFrame"""
    year_month = file_path.stem
    print(f"  -> 正在处理文件: {file_path.name}")
    try:
        df = pd.read_excel(
            file_path, sheet_name=Config.TARGET_SHEET_NAME,
            skiprows=Config.SKIP_ROWS + Config.HEADER_ROWS_COUNT,
            header=None,
            dtype=str
        )

        df.rename(columns={0: '序号', 1: '行业'}, inplace=True)
        df = df[df['行业'].astype(str).str.strip() != '栏目'].copy()
        df.dropna(how='all', subset=df.columns[2:], inplace=True)
        df['序号'] = robust_to_numeric(df['序号'])
        df.dropna(subset=['序号'], inplace=True)
        df['序号'] = df['序号'].astype(int)
        df['年月'] = year_month

        df_final = df[['序号', '行业', '年月']].copy()

        ### 重构/新增开始 ###
        # 根据新需求，提取栏目13至24的数据
        # 换算公式： Pandas索引 = Excel列号 - 1
        # 原始数据栏目号从1开始，所以 Pandas索引 = 原始栏目号 + 1 (此为原代码逻辑，我们遵循它)

        # 提取 完成新装增容 数据
        df_final[Config.COL_TOTAL_ALL_COUNT]    = robust_to_numeric(df.iloc[:, 13+1]) # 栏目13
        df_final[Config.COL_TOTAL_ALL_CAPACITY] = robust_to_numeric(df.iloc[:, 14+1]) # 栏目14
        df_final[Config.COL_TOTAL_HV_COUNT]     = robust_to_numeric(df.iloc[:, 15+1]) # 栏目15
        df_final[Config.COL_TOTAL_HV_CAPACITY]  = robust_to_numeric(df.iloc[:, 16+1]) # 栏目16
        df_final[Config.COL_TOTAL_LV_COUNT]     = robust_to_numeric(df.iloc[:, 17+1]) # 栏目17
        df_final[Config.COL_TOTAL_LV_CAPACITY]  = robust_to_numeric(df.iloc[:, 18+1]) # 栏目18

        # 提取 完成新装 数据
        df_final[Config.COL_NEW_ALL_COUNT]      = robust_to_numeric(df.iloc[:, 19+1]) # 栏目19
        df_final[Config.COL_NEW_ALL_CAPACITY]   = robust_to_numeric(df.iloc[:, 20+1]) # 栏目20
        df_final[Config.COL_NEW_HV_COUNT]       = robust_to_numeric(df.iloc[:, 21+1]) # 栏目21
        df_final[Config.COL_NEW_HV_CAPACITY]    = robust_to_numeric(df.iloc[:, 22+1]) # 栏目22
        df_final[Config.COL_NEW_LV_COUNT]       = robust_to_numeric(df.iloc[:, 23+1]) # 栏目23
        df_final[Config.COL_NEW_LV_CAPACITY]    = robust_to_numeric(df.iloc[:, 24+1]) # 栏目24
        ### 重构/新增结束 ###

        return df_final
    except Exception as e:
        print(f"    [错误] 处理文件 {file_path.name} 时发生未知错误。")
        print(traceback.format_exc())
        return None

# ==============================================================================
# 3. 主流程 (Transformation & Loading)
# ==============================================================================
def main():
    print("开始执行数据处理流程...")
    all_dataframes = []
    if not Config.INPUT_DIR.exists():
        print(f"[致命错误] 输入文件夹不存在: {Config.INPUT_DIR}")
        return
    for file_path in sorted(Config.INPUT_DIR.glob('*.xlsx')):
        if re.match(Config.FILENAME_PATTERN, file_path.name):
            df = process_single_file(file_path)
            if df is not None:
                all_dataframes.append(df)
    if not all_dataframes:
        print("\n处理结束，但未从任何文件中成功提取数据。请检查上述错误日志。")
        return
    print("\n所有文件数据读取完毕，正在聚合成总表...")
    long_df = pd.concat(all_dataframes, ignore_index=True)

    long_df.fillna(0, inplace=True)

    ### 重构/新增开始 ###
    # 在聚合后的 long_df 上执行计算，生成所有“增容”相关的衍生指标
    print("正在计算所有衍生指标 (增容 = 新装增容 - 新装)...")

    # 计算 合计增容
    long_df[Config.COL_INC_ALL_COUNT]    = long_df[Config.COL_TOTAL_ALL_COUNT] - long_df[Config.COL_NEW_ALL_COUNT]
    long_df[Config.COL_INC_ALL_CAPACITY] = long_df[Config.COL_TOTAL_ALL_CAPACITY] - long_df[Config.COL_NEW_ALL_CAPACITY]

    # 计算 高压增容
    long_df[Config.COL_INC_HV_COUNT]     = long_df[Config.COL_TOTAL_HV_COUNT] - long_df[Config.COL_NEW_HV_COUNT]
    long_df[Config.COL_INC_HV_CAPACITY]  = long_df[Config.COL_TOTAL_HV_CAPACITY] - long_df[Config.COL_NEW_HV_CAPACITY]

    # 计算 低压增容
    long_df[Config.COL_INC_LV_COUNT]     = long_df[Config.COL_TOTAL_LV_COUNT] - long_df[Config.COL_NEW_LV_COUNT]
    long_df[Config.COL_INC_LV_CAPACITY]  = long_df[Config.COL_TOTAL_LV_CAPACITY] - long_df[Config.COL_NEW_LV_CAPACITY]
    ### 重构/新增结束 ###

    print("正在生成最终的多工作表Excel报表...")
    Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(Config.OUTPUT_FILE, engine='openpyxl') as writer:
        sheets_to_create = Config.get_sheets_to_create()
        # 现在的 sheets_to_create 已经包含了全部18个指标的工作表
        for sheet_name, value_col in sheets_to_create.items():
            print(f"  -> 正在生成工作表: {sheet_name}")
            try:
                pivot_df = long_df.pivot_table(
                    index=['序号', '行业'], columns='年月', values=value_col, aggfunc='sum')
                # 填充透视表中可能出现的NaN（例如某个月份某个行业没有数据）
                pivot_df.fillna(0, inplace=True)
                pivot_df.columns.name = None
                pivot_df = pivot_df.reset_index()
                pivot_df.rename(columns={'行业': '分类'}, inplace=True)
                pivot_df.to_excel(writer, sheet_name=sheet_name, index=False)
            except KeyError as e:
                print(f"    [警告] 在生成工作表 '{sheet_name}' 时找不到数据列 '{e}'，已跳过。")
                continue
    print(f"\n全部任务成功完成！结果已保存到: {Config.OUTPUT_FILE}")

if __name__ == '__main__':
    main()